In [1]:
pip install gradio

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import gradio as gr
import numpy  as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn.linear_model import LinearRegression
from joblib import dump

In [3]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
final=pd.read_csv("test.csv")

In [4]:
df=pd.concat([train,test])
df['date']=pd.to_datetime(df['date'])


In [5]:
def cols_new(data_df):
    data_df['year'] = data_df['date'].dt.year
    data_df['quarter'] = data_df['date'].dt.quarter
    data_df['month'] = data_df['date'].dt.month
    data_df['weekofyear'] = data_df['date'].dt.weekofyear
    data_df['dayofweek'] = data_df['date'].dt.dayofweek
    return data_df

In [6]:
cols_new(df)

/tmp/ipykernel_10516/1417406636.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  data_df['weekofyear'] = data_df['date'].dt.weekofyear


,date,store,item,sales,id,year,quarter,month,weekofyear,dayofweek
0,2013-01-01,1,1,13.0,NaN,2013,1,1,1,1
1,2013-01-02,1,1,11.0,NaN,2013,1,1,1,2
2,2013-01-03,1,1,14.0,NaN,2013,1,1,1,3
3,2013-01-04,1,1,13.0,NaN,2013,1,1,1,4
4,2013-01-05,1,1,10.0,NaN,2013,1,1,1,5
...,...,...,...,...,...,...,...,...,...,...
44995,2018-03-27,10,50,NaN,44995.0,2018,1,3,13,1
44996,2018-03-28,10,50,NaN,44996.0,2018,1,3,13,2
44997,2018-03-29,10,50,NaN,44997.0,2018,1,3,13,3
44998,2018-03-30,10,50,NaN,44998.0,2018,1,3,13,4


In [7]:
%%time
def mean_cols(data,cols):
    for i in cols:
        cols=[e for e in cols if e not in (i)]
        for j in cols :
            if i!=j :
                data['mean_'+i+'_'+j]=data.groupby([i,j])['sales'].transform('mean')
    return data

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.48 µs


In [8]:
df.columns

Index(['date', 'store', 'item', 'sales', 'id', 'year', 'quarter', 'month',
       'weekofyear', 'dayofweek'],
      dtype='object')

In [9]:
%%time
mean_cols(df,['item','store','dayofweek','weekofyear','month','quarter'])
print(df.columns)

Index(['date', 'store', 'item', 'sales', 'id', 'year', 'quarter', 'month',
       'weekofyear', 'dayofweek', 'mean_item_store', 'mean_item_dayofweek',
       'mean_item_weekofyear', 'mean_item_month', 'mean_item_quarter',
       'mean_store_dayofweek', 'mean_store_weekofyear', 'mean_store_month',
       'mean_store_quarter', 'mean_dayofweek_weekofyear',
       'mean_dayofweek_month', 'mean_dayofweek_quarter',
       'mean_weekofyear_month', 'mean_weekofyear_quarter',
       'mean_month_quarter'],
      dtype='object')
CPU times: user 719 ms, sys: 650 ms, total: 1.37 s
Wall time: 1.4 s


In [10]:
def median_cols(data,cols):
    for i in cols:
        cols=[e for e in cols if e not in (i)]
        for j in cols :
            if i!=j :
                data['median_'+i+'_'+j]=data.groupby([i,j])['sales'].transform('median')
    return data

In [11]:
%%time
median_cols(df,['item','store','dayofweek','weekofyear','month','quarter'])
print(df.columns)

Index(['date', 'store', 'item', 'sales', 'id', 'year', 'quarter', 'month',
       'weekofyear', 'dayofweek', 'mean_item_store', 'mean_item_dayofweek',
       'mean_item_weekofyear', 'mean_item_month', 'mean_item_quarter',
       'mean_store_dayofweek', 'mean_store_weekofyear', 'mean_store_month',
       'mean_store_quarter', 'mean_dayofweek_weekofyear',
       'mean_dayofweek_month', 'mean_dayofweek_quarter',
       'mean_weekofyear_month', 'mean_weekofyear_quarter',
       'mean_month_quarter', 'median_item_store', 'median_item_dayofweek',
       'median_item_weekofyear', 'median_item_month', 'median_item_quarter',
       'median_store_dayofweek', 'median_store_weekofyear',
       'median_store_month', 'median_store_quarter',
       'median_dayofweek_weekofyear', 'median_dayofweek_month',
       'median_dayofweek_quarter', 'median_weekofyear_month',
       'median_weekofyear_quarter', 'median_month_quarter'],
      dtype='object')
CPU times: user 1 s, sys: 501 ms, total: 1.5 s
Wall ti

In [12]:
train = df.loc[~df.sales.isna()]
test = df.loc[df.sales.isna()]

In [13]:
X_train = train.drop(['date','sales','id'], axis=1)
y_train = train['sales'].values
X_test = test.drop(['id','date','sales'], axis=1)

In [14]:
x_train, x_validate, y_train, y_validate = train_test_split(X_train, y_train, random_state=100, test_size=0.25)

In [15]:
%%time
params = {
    'colsample_bytree': 0.8,
    'eta': 0.1,
    'eval_metric': 'mae',
    'lambda': 1,
    'max_depth': 6,
    'objective': 'reg:linear',
    'seed': 0,
    'silent': 1,
    'subsample': 0.8,
}
xgbtrain = xgb.DMatrix(x_train, label=y_train)
xgbvalidate = xgb.DMatrix(x_validate, label=y_validate)
xgbmodel = xgb.train(list(params.items()), xgbtrain, early_stopping_rounds=50,
                     evals=[(xgbtrain, 'train'), (xgbvalidate, 'validate')], 
                     num_boost_round=200, verbose_eval=50)

[21:06:31] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[21:06:31] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.

[0]	train-mae:46.58235	validate-mae:46.55130
[50]	train-mae:5.57271	validate-mae:5.58496
[100]	train-mae:5.55506	validate-mae:5.58027
[150]	train-mae:5.54345	validate-mae:5.58059
[159]	train-mae:5.54160	validate-mae:5.58058
CPU times: user 14min 7s, sys: 3.48 s, total: 14min 10s
Wall time: 4min 6s


In [16]:
print(test.columns)
print(final.columns)

Index(['date', 'store', 'item', 'sales', 'id', 'year', 'quarter', 'month',
       'weekofyear', 'dayofweek', 'mean_item_store', 'mean_item_dayofweek',
       'mean_item_weekofyear', 'mean_item_month', 'mean_item_quarter',
       'mean_store_dayofweek', 'mean_store_weekofyear', 'mean_store_month',
       'mean_store_quarter', 'mean_dayofweek_weekofyear',
       'mean_dayofweek_month', 'mean_dayofweek_quarter',
       'mean_weekofyear_month', 'mean_weekofyear_quarter',
       'mean_month_quarter', 'median_item_store', 'median_item_dayofweek',
       'median_item_weekofyear', 'median_item_month', 'median_item_quarter',
       'median_store_dayofweek', 'median_store_weekofyear',
       'median_store_month', 'median_store_quarter',
       'median_dayofweek_weekofyear', 'median_dayofweek_month',
       'median_dayofweek_quarter', 'median_weekofyear_month',
       'median_weekofyear_quarter', 'median_month_quarter'],
      dtype='object')
Index(['id', 'date', 'store', 'item'], dtype='object')

In [17]:
model = xgbmodel
predict=model.predict(xgb.DMatrix(X_test),ntree_limit=model.best_ntree_limit)
dump(xgbmodel, 'xgbmodel_model.joblib')
final['sales'] = np.round(predict)

/home/susmitha/.local/lib/python3.10/site-packages/xgboost/core.py:122: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


In [18]:
# def predict_sales(date,store,item):
#     # sample= np.array([id,sales]).reshape(1,-1)
#     model = xgbmodel

#     predict=model.predict(xgb.DMatrix(X_test),ntree_limit=model.best_ntree_limit)
#     print(predict)
#     final['sales'] = np.round(predict)
#     ans = final.groupby(['item','store','date'])['sales'].mean()
#     return (ans.loc[(item, store, date)])

In [19]:
def predict_sales(date,store,item):
    # sample= np.array([id,sales]).reshape(1,-1)    
    ans = final.groupby(['item','store','date'])['sales'].mean()
    return (ans.loc[(item, store, date)])

In [20]:
input_components = [
    gr.inputs.Textbox(label="Date(YYYY-MM-DD)"),
    gr.inputs.Number(label="Store"),
    gr.inputs.Number(label="Item")
]

/home/susmitha/.local/lib/python3.10/site-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/home/susmitha/.local/lib/python3.10/site-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/home/susmitha/.local/lib/python3.10/site-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
/home/susmitha/.local/lib/python3.10/site-packages/gradio/inputs.py:59: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(


In [21]:
interface = gr.Interface(
    fn=predict_sales,
    inputs=input_components,
    outputs=gr.outputs.Textbox(label="Sales prediction"),
    title="Predicting sales",
    description="Predicting 3 months of sales given 5 year data",
    layout="vertical",
    theme="compact"
)

/home/susmitha/.local/lib/python3.10/site-packages/gradio/outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
/home/susmitha/.local/lib/python3.10/site-packages/gradio/blocks.py:528: UserWarning: Cannot load compact. Caught Exception: The space compact does not exist
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")
/home/susmitha/.local/lib/python3.10/site-packages/gradio/deprecation.py:40: UserWarning: `layout` parameter is deprecated, and it has no effect
  warnings.warn(value)


In [22]:
interface.launch(debug = False,share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://d5cf67d217231cdec3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
